Assignment: Flowers Recognition <br>
Dataset Description:<br>

This dataset contains 4242 images of flowers.<br>
The data collection is based on the data flicr, google images, yandex images.<br>
You can use this datastet to recognize plants from the photo.<br>

Attribute Information:<br>
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. <br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size. </b> <br>
This is a Multiclass Classification Problem.<br>




WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>




Data : <br>
https://drive.google.com/file/d/1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj/view?usp=sharing

In [1]:
import glob
import cv2
import numpy as np

IMG_DIR='./daisy'
IMG_DIR1='./dandelion'
IMG_DIR2='./rose'
IMG_DIR3='./sunflower'
IMG_DIR4='./tulip'

def read_images(directory):
    for img in glob.glob(directory+"/*.jpg"):
        image = cv2.imread(img)
        resized_img = cv2.resize(image/255.0  , (150 , 150))

        yield resized_img

resized_imgs0 =  np.array(list(read_images(IMG_DIR)))
resized_imgs1 =  np.array(list(read_images(IMG_DIR1)))
resized_imgs2 =  np.array(list(read_images(IMG_DIR2)))
resized_imgs3 =  np.array(list(read_images(IMG_DIR3)))
resized_imgs4 =  np.array(list(read_images(IMG_DIR4)))

In [2]:
label0 = np.zeros((resized_imgs0.shape[0],1))   #0
label1 = np.ones((resized_imgs1.shape[0],1))  #1
label2 = np.ones((resized_imgs2.shape[0],1))*2 #2
label3 = np.ones((resized_imgs3.shape[0],1))*3 #3
label4 = np.ones((resized_imgs4.shape[0],1))*4  #4

In [2]:
data = np.concatenate((resized_imgs0,resized_imgs1,resized_imgs2,resized_imgs3,resized_imgs4))

In [4]:
del resized_imgs0 
del resized_imgs1 
del resized_imgs2 
del resized_imgs3 
del resized_imgs4


In [5]:
labels = np.concatenate((label0,label1,label2,label3,label4))

In [6]:
del label0
del label1
del label2
del label3
del label4

In [7]:
labels

array([[0.],
       [0.],
       [0.],
       ...,
       [4.],
       [4.],
       [4.]])

In [3]:
data.shape

(4323, 240, 320, 3)

In [8]:
# del 

In [9]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [10]:
train_data,train_labels  = unison_shuffled_copies(data,labels)

In [11]:
# train_data

In [12]:
# train_labels.shape

In [13]:
del data
del labels

In [14]:
np.random.seed(1234)
mask = np.random.rand(len(train_data)) < 0.7
train = train_data[mask]
test_data = train_data[~mask]
label = train_labels[mask]
test_label = train_labels[~mask]

In [15]:
# np.random.seed(1234)
# mask = np.random.rand(len(train_data)) < 0.7
# train = train_data[mask]
# test_data = train_data[~mask]
# label = train_labels[mask]
# test_label = train_labels[~mask]

In [16]:
train = train.reshape(3004,230400)

In [17]:
test_data = test_data.reshape(test_data.shape[0], test_data.shape[1]*test_data.shape[2]*test_data.shape[3])

In [18]:
label = label.reshape(3004,)

In [19]:
label.shape

(3004,)

In [20]:
label[0]

2.0

In [21]:
train.shape

(3004, 230400)

In [22]:
from keras.utils import to_categorical
train_labels = to_categorical(label)
test_labels = to_categorical(test_label)

In [24]:
from keras import models
from keras import layers
def model_make():
    model = models.Sequential()
    model.add(layers.Dense(10, activation='relu', input_shape=(230400,)))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))
#     model.fit_generator()
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
    return model;

In [ ]:
k = 4
num_val_samples = len(train) //4

print(num_val_samples)
num_epochs = 10
all_scores = []
for i in range(k):
    print('processing fold #', i)
    val_data = train[i * num_val_samples: (i + 1) * num_val_samples]
    print(val_data.shape)
    val_targets = label[i * num_val_samples: (i + 1) * num_val_samples]
    print(val_targets.shape)
    partial_train_data = np.concatenate([train[:i * num_val_samples],train[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = np.concatenate([train_labels[:i * num_val_samples],train_labels[(i + 1) * num_val_samples:]],axis=0)
    print(partial_train_data.shape)
    print(partial_train_targets.shape)
    model = model_make()
    model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=20, verbose=1)
    loss, val_accuracy = model.evaluate(val_data, val_targets, verbose=1)
    all_scores.append(val_acc)

751
processing fold # 0
(751, 230400)
(751,)
(2253, 230400)
(2253, 5)
Epoch 1/10
113/113 [==============================] - 5s 37ms/step - loss: 5.6183 - accuracy: 0.2226
Epoch 2/10
113/113 [==============================] - 4s 37ms/step - loss: 1.6157 - accuracy: 0.2521
Epoch 3/10
113/113 [==============================] - 4s 40ms/step - loss: 1.6378 - accuracy: 0.2395
Epoch 4/10
113/113 [==============================] - 5s 41ms/step - loss: 1.5957 - accuracy: 0.2547
Epoch 5/10
113/113 [==============================] - 5s 41ms/step - loss: 1.5949 - accuracy: 0.2492
Epoch 6/10
113/113 [==============================] - 5s 43ms/step - loss: 1.5976 - accuracy: 0.2486
Epoch 7/10
113/113 [==============================] - 5s 46ms/step - loss: 1.5909 - accuracy: 0.2597
Epoch 8/10
113/113 [==============================] - 5s 47ms/step - loss: 1.5933 - accuracy: 0.2576
Epoch 9/10
113/113 [==============================] - 5s 47ms/step - loss: 1.5935 - accuracy: 0.2386
Epoch 10/10
 86/113 [

In [ ]:
# x_val = train[:1000]
# partial_x_train = train[1000:]
# y_val = train_labels[:1000]
# partial_y_train = train_labels[1000:]

In [ ]:
print(partial_x_train.shape)

In [ ]:
print(partial_y_train.shape)

In [ ]:
# history = model.fit(partial_x_train,
# partial_y_train,
# epochs=20,
# batch_size=512,
# validation_data=(x_val, y_val))